## There are numerous decisions for me to make (very quickly):

* How to present results of classifier so that they do not bias viewer's interpretation of greyscale images. (If I use color, viewer is likely as not to be tricked by the colors into believing there are differences where labeled, even if not merited by the underlying data.
* Should I attempt iForest anomaly detection on images? **Yes!**
  * Advantage: (there exists code) [there exists code](http://stackoverflow.com/questions/30080491/isolation-forest-algorithm-in-python "code provided by user on stackoverflow")
  * Disadvantages: author claims that it is slow, it is not well tested. *Seems to work.*
  * That same reference points to sklearn github pull-request for [a model of increasing maturity](https://github.com/scikit-learn/scikit-learn/pull/4163 "not yet ready for sklearn"), but not yet fully working and not super-well-tested. Another month or two?
  * To impliment iForest, would need to extract feature weights from trained model. Can I do this with Dato's imagenet classifier? **Yes! However, there are devils in the details to be worked out.**
  
### Regarding presentation:
  
  * For minimum viable product:
    * Use [Flask-Uploads](https://pythonhosted.org/Flask-Uploads/) to allow a user to upload a directory filled with images.
    * Apply iForest and rank by anomaly score
    * Present a table with columns [256 x 256 images, file names, anomaly score], ranked by anomaly score.

  * If can't work out NN features, do minimum minimum viable product using iForest on pixel values.
  
### Things to implement:

  * Make sure that can work with color photos as well as grayscale. Should app to a detect on each file and convert to that which is most consistent with others in directory?
  * Would be great to add audio, converting to FFT, clipping or repeating to a nominal length.
  * 

In [55]:
import random
import time
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt

from code import iForest as isof

% matplotlib inline

### try iForest using real image data (raw pixels):

In [59]:
import cv2
import os
import glob
WORK_DIR = '/home/wilber/work/Galvanize/gcp-data/iForest/'
pngCt = len(glob.glob1(WORK_DIR,"*.png"))
print "pngCt: {0}".format(pngCt)
%cd /home/wilber/work/Galvanize/gcp-data/iForest/
Xraw = np.full((pngCt, 10000), np.nan)
names = [""]*pngCt
for i, file in enumerate(os.listdir(WORK_DIR)):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    if i % 100 == 0:
        print "{0}\t{1}\t{2}".format(i, file, type(img))
    Xraw[i,:] = img.flatten()
    names[i] = file
print Xraw.shape
print Xraw[:10, :20]
names[:10]

pngCt: 2064
/home/wilber/work/Galvanize/gcp-data/iForest
0	hdll020+0300+0126.png	<type 'numpy.ndarray'>
100	wshdll020+0740+0086.png	<type 'numpy.ndarray'>
200	ldlm020+0620+0086.png	<type 'numpy.ndarray'>
300	hdmm000+0460+0166.png	<type 'numpy.ndarray'>
400	nthdmm020+0340+0166.png	<type 'numpy.ndarray'>
500	wshdll000+1500+0126.png	<type 'numpy.ndarray'>
600	hdll020+0220+0086.png	<type 'numpy.ndarray'>
700	wshdmm020+1180+0166.png	<type 'numpy.ndarray'>
800	ntldlm020+0860+0166.png	<type 'numpy.ndarray'>
900	ldlm020+0780+0166.png	<type 'numpy.ndarray'>
1000	ldlm020+1620+1086.png	<type 'numpy.ndarray'>
1100	ldlm000+1060+0166.png	<type 'numpy.ndarray'>
1200	hdlr020+0300+0166.png	<type 'numpy.ndarray'>
1300	nthdlr020+0500+0166.png	<type 'numpy.ndarray'>
1400	ntldlm020+1140+1046.png	<type 'numpy.ndarray'>
1500	wshdlr000+0540+0166.png	<type 'numpy.ndarray'>
1600	hdll020+0460+0126.png	<type 'numpy.ndarray'>
1700	wsldlm000+1460+0126.png	<type 'numpy.ndarray'>
1800	hdlr020+1300+1006.png	<type 'num

['hdll020+0300+0126.png',
 'wsldlm000+0820+0166.png',
 'nthdmm000+1020+1006.png',
 'ntldlm000+1540+0166.png',
 'ntldlm000+1020+0166.png',
 'nthdlr020+0260+0046.png',
 'ntldlm020+1020+1086.png',
 'nthdll020+1340+1086.png',
 'wshdll020+0500+0086.png',
 'wshdlr000+1380+0126.png']

In [12]:
%pwd

u'/home/wilber/work/Galvanize/gcp'

### High-level features from Dato's ImageNet classifier:

In [38]:
import graphlab as gl
WORKING_DIR = '/home/wilber/work/Galvanize/gcp-data/iForest/'
data = gl.image_analysis.load_images(WORKING_DIR, \
                                     random_order=True)
data['image'] = gl.image_analysis.resize(data['image'], 256, 256)

# Import data from MNIST
#data = gl.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train6k')

# Create a DeepFeatureExtractorObject

start = time.time()
extractor = gl.feature_engineering.DeepFeatureExtractor(feature = 'image')
print help(extractor)
# Fit the encoder for a given dataset.
time1 = time.time()
print "\n\n", time1 - start
extractor = extractor.fit(data)
time2 = time.time()
print "\n\n", time2 - time1

# Return the model used for the deep feature extraction.
extracted_model = extractor['model']
time3 = time.time()
print "\n\n", time3 - time2

# Extract features.
features_sf = extractor.transform(data)
features_sf.head()
time4 = time.time()
print "\n\n", time4 - time3, ", total time: ", time4 - start

PROGRESS: Unsupported image format. Supported formats are JPG and PNG	 file: /home/wilber/work/Galvanize/gcp-data/iForest/mean/mean_array/m_3fe07a27219448.sidx
PROGRESS: Unsupported image format. Supported formats are JPG and PNG	 file: /home/wilber/work/Galvanize/gcp-data/iForest/mean/mean_array/dir_archive.ini
PROGRESS: Unsupported image format. Supported formats are JPG and PNG	 file: /home/wilber/work/Galvanize/gcp-data/iForest/mean/mean_array/m_3fe07a27219448.0000
PROGRESS: Unsupported image format. Supported formats are JPG and PNG	 file: /home/wilber/work/Galvanize/gcp-data/iForest/mean/mean_array/objects.bin
Help on DeepFeatureExtractor in module graphlab.toolkits.feature_engineering._deep_feature_extractor object:

class DeepFeatureExtractor(graphlab.toolkits.feature_engineering._feature_engineering.TransformerBase)
 |  Takes an input dataset, propagates each example through the network,
 |  and returns an SArray of dense feature vectors, each of which is the
 |  concatenation

In [41]:
#feature = features_sf.head['deep_features_image']
features_sf.num_rows(), features_sf.num_cols()

(2064, 3)

In [42]:
extractor['model']

Class               : NeuralNetClassifier

Schema
------
Examples            : 1281166
Features            : 1
Target column       : label

Training Summary
----------------
Training accuracy   : 0.625
Validation accuracy : None
Training recall@5   : 0.8512
Validation recall@5 : None
Training time (sec) : 288884.0298

In [43]:
opts_dict = extractor.get_current_options()
for key, value in opts_dict.iteritems():
    print key, value

model Class               : NeuralNetClassifier

Schema
------
Examples            : 1281166
Features            : 1
Target column       : label

Training Summary
----------------
Training accuracy   : 0.625
Validation accuracy : None
Training recall@5   : 0.8512
Validation recall@5 : None
Training time (sec) : 288884.0298



In [39]:
repr(extractor)

'Class               : DeepFeatureExtractor\n\nModel fields\n------------\nFeature(s)          : image\nOutput Column Name  : deep_features_image\n'

In [13]:
features_sf.column_names()

['path', 'image', 'deep_features_image']

In [15]:
np.shape(features_sf['deep_features_image'])

(2064, 4096)

In [19]:
X = np.array(features_sf['deep_features_image'])

In [32]:
X[:5]

array([[ 0.        ,  3.72930121,  0.        , ...,  0.        ,
         0.94428813,  0.        ],
       [ 0.        ,  1.59159303,  0.        , ...,  0.        ,
         0.6683253 ,  0.60917962],
       [ 1.48337448,  0.        ,  0.65555978, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  2.86736774,  0.        , ...,  0.        ,
         1.75487328,  0.        ],
       [ 1.57783175,  1.64519286,  0.        , ...,  0.        ,
         1.5495981 ,  0.        ]])

In [62]:
model = isof.iForest(n_estimators=200)
start = time.time()
model.fit(X)
end = time.time()
print end - start

369.748157024


In [51]:
anom_scores = model.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	wsldlm000+1340+0126.png	0.615487381279
2	nthdll000+0820+0166.png	0.590250183857
3	hdlr020+0980+0166.png	0.566325593441
4	nthdlr020+1100+1046.png	0.549250004206
5	hdmm020+1020+0166.png	0.488050215793
6	ldlm020+0300+0166.png	0.474663087353
7	wshdmm020+0900+0086.png	0.459938195728
8	nthdll020+1180+1126.png	0.455982553333
9	wshdll020+0420+0086.png	0.452169702796
10	hdmm020+0340+0126.png	0.451372111184
11	nthdmm020+0820+1006.png	0.448498633964
12	wshdmm020+0620+0126.png	0.446066758839
13	hdlr020+0540+0046.png	0.441880783274
14	ntldlm000+0300+1006.png	0.440740666117
15	hdll000+0460+0166.png	0.440144821711


478	fig01.png	0.322530355659
737	fig03.png	0.301604729085
1165	fig00.png	0.275853525065
1850	fig02.png	0.234054963318


### Different layers from pre-trained model:

#### Layer 22

In [84]:
start = time.time()
pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
time1 = time.time()
print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 22)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

0.601696014404
803.75030899
Elapsed time = 0 hours, 13 minutes, 24.3527450562 seconds


In [86]:
X22 = np.array(data['extracted_features'])
print np.shape(X22)
X22[:5]

(2064, 1000)


array([[ 0.39133096, -0.27442911, -0.64612275, ..., -0.02209489,
         1.34912241, -1.28404224],
       [ 1.76750171,  0.5814544 ,  0.55570209, ..., -0.29647955,
         1.31140757, -3.0269506 ],
       [ 1.33953321,  0.03642526,  0.26932773, ..., -0.57074183,
         1.61481857, -1.73673403],
       [ 2.4980371 ,  0.22236168,  5.77681351, ..., -2.55784583,
         0.7607308 , -0.28638375],
       [ 1.62160802, -1.22865927, -0.52635783, ..., -0.98451412,
         2.88381124, -3.38303924]])

In [87]:
m22 = isof.iForest(n_estimators=200)
start = time.time()
m22.fit(X22)
end = time.time()
print end - start

10.8693459034


In [88]:
anom_scores = m22.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	nthdmm020+0940+1046.png	0.703939190143
2	ldlm000+0220+0166.png	0.672911750932
3	ldlm020+0300+0086.png	0.656170520923
4	nthdlr020+1380+1166.png	0.653061031107
5	wsldlm000+0500+0166.png	0.598530571757
6	wshdmm000+0660+0166.png	0.598298523198
7	nthdll000+1060+1006.png	0.58320996648
8	hdmm020+0420+0166.png	0.56711221415
9	nthdlr020+1340+1126.png	0.564080096198
10	wshdll000+1300+0166.png	0.558592832075
11	wshdlr000+1380+0126.png	0.556089810145
12	wsldlm000+1580+0126.png	0.553821170925
13	ntldlm020+0380+0086.png	0.549241340939
14	wsldlm020+1620+1006.png	0.549118802039
15	nthdmm000+0740+1006.png	0.539126716373


691	fig01.png	0.426010137662
979	fig03.png	0.40839662886
981	fig00.png	0.408277968147
1170	fig02.png	0.397979438503


### layer 21

In [67]:
start = time.time()
pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
time1 = time.time
print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 21)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

Help on NeuralNetClassifier in module graphlab.toolkits.classifier.neuralnet_classifier object:

neuralnet_classifier_v2 = class NeuralNetClassifier(graphlab.toolkits._supervised_learning.Classifier)
 |  Neural Network is one of the classical models in artificial intelligence and
 |  machine learning, and has recently achieved great success in computer vision
 |  tasks such as object recognition.
 |  
 |  This model cannot be constructed directly.  Instead, use
 |  :func:`graphlab.neuralnet_classifier.create` to create an instance of
 |  this model.
 |  Additional details on parameter options and code samples are available in
 |  the documentation for the create function.
 |  
 |  See Also
 |  --------
 |  create
 |  graphlab.deeplearning.NeuralNet
 |  graphlab.deeplearning.create
 |  
 |  Examples
 |  --------
 |  >>> # Load the data (From an S3 bucket)
 |  >>> data = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train6k')
 |  >>> test_data = graphlab.SFrame('htt

In [83]:
X21 = np.array(data['extracted_features'])
print np.shape(X21)
X21[:5]

(2064, 4096)


array([[ 0.13311839,  1.74991465, -0.53586638, ..., -4.63365936,
         1.16232181, -2.3071599 ],
       [ 0.34298587,  2.45043421,  0.31843156, ..., -3.06454062,
         0.06621611, -2.48183918],
       [-0.66870165,  2.05958056, -0.98619831, ..., -4.17561054,
         1.24312103, -2.19206047],
       [-1.57564211,  2.51387453, -1.62678885, ..., -2.4753325 ,
        -0.29885268, -2.04687071],
       [-3.85620332,  6.15582085, -4.85414505, ..., -4.13845158,
        -0.13795936,  1.29018736]])

In [69]:
m21 = isof.iForest(n_estimators=200)
start = time.time()
m21.fit(X21)
end = time.time()
print end - start

36.6877818108


In [70]:
anom_scores = m21.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	ldlm020+0300+0086.png	0.612971364204
2	nthdmm020+0940+1046.png	0.606459703844
3	ldlm000+0220+0166.png	0.569214162744
4	nthdlr020+1380+1166.png	0.544138268275
5	wshdlr000+1380+0126.png	0.465919982239
6	wshdmm000+0660+0166.png	0.465663325207
7	hdmm020+1380+1046.png	0.465116244784
8	hdll020+1620+1086.png	0.452411784979
9	hdll020+0300+0086.png	0.451601639546
10	hdlr020+1380+1046.png	0.451599022895
11	hdll020+0820+0086.png	0.449576931246
12	nthdmm000+0820+1006.png	0.448086825148
13	wshdmm000+1140+0166.png	0.445078425141
14	nthdlr020+0860+1006.png	0.443996206774
15	wshdll020+0740+0166.png	0.442248881515


646	fig01.png	0.307947245958
1049	fig03.png	0.281181795939
1261	fig02.png	0.268989787051
1361	fig00.png	0.264065256583


#### Layer 20

In [72]:
start = time.time()
pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
time1 = time.time()
print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 20)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

0.530540943146
Help on NeuralNetClassifier in module graphlab.toolkits.classifier.neuralnet_classifier object:

neuralnet_classifier_v2 = class NeuralNetClassifier(graphlab.toolkits._supervised_learning.Classifier)
 |  Neural Network is one of the classical models in artificial intelligence and
 |  machine learning, and has recently achieved great success in computer vision
 |  tasks such as object recognition.
 |  
 |  This model cannot be constructed directly.  Instead, use
 |  :func:`graphlab.neuralnet_classifier.create` to create an instance of
 |  this model.
 |  Additional details on parameter options and code samples are available in
 |  the documentation for the create function.
 |  
 |  See Also
 |  --------
 |  create
 |  graphlab.deeplearning.NeuralNet
 |  graphlab.deeplearning.create
 |  
 |  Examples
 |  --------
 |  >>> # Load the data (From an S3 bucket)
 |  >>> data = graphlab.SFrame('http://s3.amazonaws.com/dato-datasets/mnist/sframe/train6k')
 |  >>> test_data = graph

In [73]:
X20 = np.array(data['extracted_features'])
print np.shape(X20)
X20[:5]

(2064, 4096)


array([[ 0.13311839,  1.74991465,  0.        , ...,  0.        ,
         1.16232181,  0.        ],
       [ 0.34298587,  2.45043421,  0.31843156, ...,  0.        ,
         0.06621611,  0.        ],
       [ 0.        ,  2.05958056,  0.        , ...,  0.        ,
         1.24312103,  0.        ],
       [ 0.        ,  2.51387453,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  6.15582085,  0.        , ...,  0.        ,
         0.        ,  1.29018736]])

In [74]:
m20 = isof.iForest(n_estimators=200)
start = time.time()
m20.fit(X20)
end = time.time()
print end - start

37.6528069973


In [75]:
anom_scores = m20.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	nthdmm020+0940+1046.png	0.595912931591
2	ldlm000+0220+0166.png	0.575106881181
3	ldlm020+0300+0086.png	0.565322775139
4	nthdlr020+1380+1166.png	0.559275424649
5	wshdmm000+0660+0166.png	0.508471408203
6	wshdlr000+1380+0126.png	0.491321908729
7	hdmm020+1380+1046.png	0.487924836722
8	wshdlr000+0180+0166.png	0.483795201816
9	hdmm020+0940+0166.png	0.480773542299
10	wshdmm000+1140+0166.png	0.470660062073
11	hdmm020+0420+0166.png	0.468624072668
12	hdll020+1620+1086.png	0.467303049218
13	wshdll020+0740+0166.png	0.460842271418
14	hdlr020+1380+1046.png	0.45796698223
15	nthdll020+1180+1126.png	0.457491335966


731	fig03.png	0.306764515088
856	fig01.png	0.298442299622
1196	fig00.png	0.279005151003
1223	fig02.png	0.277403725868


#### Layer 19

In [ ]:
start = time.time()
pretrained_model = gl.load_model('http://s3.amazonaws.com/dato-datasets/deeplearning/imagenet_model_iter45')
time1 = time.time()
print time1 - start
data['extracted_features'] = pretrained_model.extract_features(data, 19)
print time.time() - time1
totsecs = time.time() - start
hours = int(totsecs/3600)
mins = int((totsecs - 3600.*hours)/60)
secs = totsecs - 3600.*hours - 60.*mins
print "Elapsed time = {0} hours, {1} minutes, {2} seconds".format(hours, mins, secs)

In [ ]:
X19 = np.array(data['extracted_features'])
print np.shape(X19)
X19[:5]

In [ ]:
m19 = isof.iForest(n_estimators=200)
start = time.time()
m19.fit(X19)
end = time.time()
print end - start

In [85]:
anom_scores = m19.anomaly_score_
sort_indices = np.argsort(anom_scores)
for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	wshdlr020+0940+0086.png	0.738360641833
2	ntldlm020+0380+0086.png	0.661715994829
3	hdll000+0580+0166.png	0.657598281129
4	hdll020+0420+0166.png	0.645117571722
5	ldlm000+0220+0166.png	0.64409745037
6	wshdmm020+0540+0126.png	0.640095443936
7	hdlr020+0940+0126.png	0.630928611102
8	nthdmm000+0780+1006.png	0.627408515018
9	nthdmm020+0940+1046.png	0.614622902864
10	wshdlr000+1020+0126.png	0.611898833369
11	nthdmm000+0180+1006.png	0.609212665349
12	nthdmm020+0660+0086.png	0.604429025503
13	nthdlr020+1380+1166.png	0.599252722627
14	ntldlm000+1020+0166.png	0.598826325631
15	wshdlr020+0820+0046.png	0.597980794482


551	fig03.png	0.417132264575
1065	fig00.png	0.383883846599
1437	fig02.png	0.365714986349
1771	fig01.png	0.345910483029


### Now try with raw pixels:

### Sux. Will suggests mean image subtraction as a possible issue.

*A good suggestion, but Dato says not needed.*

In [81]:
model = isof.iForest(n_estimators=200)
start = time.time()
model.fit(Xraw)
end = time.time()
print end - start

93.4639201164


In [82]:
anom_scores = model.anomaly_score_
sort_indices = np.argsort(anom_scores)

for i in range(1,16):
    ind = sort_indices[-i]
    print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

print "\n"
for i in range(1, 2000):
    ind = sort_indices[-i]
    if names[ind].startswith('fig'):
        print "{0}\t{1}\t{2}".format(i, names[ind], anom_scores[ind])

1	fig03.png	0.642686526162
2	fig01.png	0.592166616477
3	fig00.png	0.584799065274
4	hdll020+1220+1006.png	0.571184821503
5	hdll020+1260+1006.png	0.57079325063
6	nthdll020+1300+1166.png	0.568242523467
7	hdll020+1460+1086.png	0.560627337362
8	nthdll020+0780+1006.png	0.555812389857
9	nthdll020+1100+1086.png	0.554378542823
10	nthdll000+0980+1006.png	0.548041106224
11	nthdll020+0820+1006.png	0.545517864877
12	hdll020+1300+1006.png	0.544984478374
13	nthdll020+1260+1126.png	0.54364769456
14	nthdll000+0900+1006.png	0.542881674059
15	hdll020+1500+1086.png	0.541418920174


1	fig03.png	0.642686526162
2	fig01.png	0.592166616477
3	fig00.png	0.584799065274
63	fig02.png	0.489604189264
